In [1]:
import pandas as pd
import numpy as np
import nltk
# Cleaning the texts
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from imblearn.over_sampling import RandomOverSampler
from joblib import dump, load
import pickle

In [2]:

#from google.colab import files 
#import io
#uploaded = files.upload()
#data = io.BytesIO(uploaded['1000 leads.xlsx'])    

# New Section

In [3]:
data = pd.read_excel("1000 leads.xlsx")

In [4]:
data.drop(['Lead Name', 'Unnamed: 4'],axis =1,inplace=True)

In [5]:
data.head()
data.columns

data.isnull().sum()

Location              21
Status                 3
Status information    24
dtype: int64

In [6]:
data_new=data.dropna()

In [7]:
data_new.isnull().sum()

Location              0
Status                0
Status information    0
dtype: int64

In [8]:
data_new.reset_index(drop=True, inplace=True)

In [9]:
data_new = data_new.rename(columns={'Status information': 'Status_information'})
data_new = data_new.rename(columns={'Status ': 'Status'})
data.columns

Index(['Location', 'Status ', 'Status information'], dtype='object')

In [10]:
data_new.replace("[^a-zA-Z]"," ",regex=True, inplace=True)

In [11]:
import string
data_lower=data_new.apply(lambda x: x.astype(str).str.lower())
data_lower.head()

,Location,Status,Status_information
0,hyderabad,not converted,prema share me details available in evn...
1,pune,not converted,prema cal me tmrw shared details to ema...
2,bangalore,converted,aug mohan rnr
3,mumbai,not converted,surendra i want only server aug mo...
4,hyderabad,not converted,prema rnr prema gave info he said ...


In [12]:
data_lower.apply(lambda x: x.astype(str).str.strip())
#df1['State'] = df1['State'].str.strip()

,Location,Status,Status_information
0,hyderabad,not converted,prema share me details available in evng ...
1,pune,not converted,prema cal me tmrw shared details to email ...
2,bangalore,converted,aug mohan rnr
3,mumbai,not converted,surendra i want only server aug mohan cal ...
4,hyderabad,not converted,prema rnr prema gave info he said he wi...
...,...,...,...
951,bangalore,not converted,surendra please send me details tomorrow afte...
952,chennai,not converted,prema need ct but our venue is far shared ...
953,bangalore,not converted,apr mohan intrstd in evng batch trail perd ...
954,bangalore,not converted,surendra please share me details gowt...


In [13]:
data_lower['Status'].unique()

array(['not converted', 'converted ', 'conveted'], dtype=object)

In [14]:
data_lower['Status']= data_lower['Status'].replace(['conveted'],'converted ')
data_lower['Status'].unique()

array(['not converted', 'converted '], dtype=object)

In [15]:
data_x=data_lower.drop(['Status'],axis =1)
data_x.head()

,Location,Status_information
0,hyderabad,prema share me details available in evn...
1,pune,prema cal me tmrw shared details to ema...
2,bangalore,aug mohan rnr
3,mumbai,surendra i want only server aug mo...
4,hyderabad,prema rnr prema gave info he said ...


In [16]:

data_lower['Status']=data_lower['Status'].map({'converted ':0,'not converted':1})
Y=data_lower.drop(['Location','Status_information'],axis =1)
Y.head()


,Status
0,1
1,1
2,0
3,1
4,1


In [17]:
ros = RandomOverSampler(random_state=0)
X_res,Y_res=ros.fit_resample(data_x,Y)

In [18]:
X_res.head()

,Location,Status_information
0,hyderabad,prema share me details available in evn...
1,pune,prema cal me tmrw shared details to ema...
2,bangalore,aug mohan rnr
3,mumbai,surendra i want only server aug mo...
4,hyderabad,prema rnr prema gave info he said ...


In [19]:
Status_information = []
for row in range(0,len(X_res.index)):
    Status_information.append(' '.join(str(x) for x in X_res.iloc[row,0:2]))

In [20]:
Status_information[955]


'bangalore         prema   shared details need only cert          prema   need to call at  pm in a meeting will cal later         prema   rnr        prema   will get back to u         gowtham tim nedded          gowtham rnr        gowtham rnr        prema rnr        prema   shared itil intermediate details         prema rnr         prema rnr         prema rnr         prema   i m in a meeting cal u back later         prema   rnr'

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [22]:
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(Status_information)

pickle.dump(cv,open('transform_new.pkl','wb'))

In [23]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf=TfidfTransformer()

X=tfidf.fit_transform(X)

In [24]:
print(X.shape,Y_res.shape)

(1658, 1246) (1658, 1)


In [25]:
pickle.dump(tfidf,open('transform.pkl','wb'))

In [26]:
# Train Test Split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y_res, test_size = 0.2, random_state = 42)

In [27]:
# implement RandomForest Classifier
clf=RandomForestClassifier(n_estimators=200,criterion='entropy')
clf.fit(X_train,y_train)


E:\Anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(criterion='entropy', n_estimators=200)

In [28]:
y_pred=clf.predict(X_test)

In [29]:
from sklearn import metrics

print(metrics.classification_report(y_test,y_pred))
print(metrics.confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97       156
           1       0.99      0.95      0.97       176

    accuracy                           0.97       332
   macro avg       0.97      0.97      0.97       332
weighted avg       0.97      0.97      0.97       332

[[154   2]
 [  8 168]]


In [30]:
y_pred

array([0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0,

In [31]:
filename='nlp_model_new.pkl'
pickle.dump(clf,open(filename,'wb'))
